In [37]:
import pandas as pd
import zipfile

### Initial Data Sourcing and Ingestion

For the NCUA data, we sourced this via a download of a text repository of credit union call report data for a given time period This can be found at the below URL:
* https://ncua.gov/analysis/credit-union-corporate-call-report-data/quarterly-data

For the FDIC data, we sourced this via interfacing with the FDIC data API, which is done via a custom URL string. While we construct our custom URL string in this notebook, the instructions for accessing the API can be found at the below URL:
* https://banks.data.fdic.gov/docs/All%20Financial%20Reports.xlsx

### NCUA Data

In [38]:
#define paths to each file within NCUA zip data download
NCUApath = 'call-report-data-2024-06.zip'
NCUADescPath = 'AcctDesc.txt'
FS220PPath = 'FS220P.txt'
FS220Path = 'FS220.txt'
FS220NPath = 'FS220N.txt'
FS220APath = 'FS220A.txt'
FOICUPath = 'FOICU.txt'

with zipfile.ZipFile(NCUApath, 'r') as z:
    with z.open(NCUADescPath) as f:
        NCUAdesc = pd.read_csv(f, delimiter=",")

    with z.open(FS220PPath) as f:
        FS220P = pd.read_csv(f, delimiter=",")

    with z.open(FS220Path) as f:
        FS220 = pd.read_csv(f, delimiter=",")

    with z.open(FS220NPath) as f:
        FS220N = pd.read_csv(f, delimiter=",")

    with z.open(FS220APath) as f:
        FS220A = pd.read_csv(f, delimiter=",")

    with z.open(FOICUPath) as f:
        FOICU = pd.read_csv(f, delimiter=",")

In [39]:
#Create list of NCUA accounts of interest for analysis - based on review of call report sample
NCUAIntAcctsList = ['ACCT_AS0009', 'ACCT_AS0013', 'Acct_025B', 'Acct_010', 'Acct_LI0069', 'Acct_014', 'Acct_115', 'ACCT_IS0010', 'Acct_117', 'Acct_671',
 'Acct_661A', 'Acct_083', 'Acct_452', 'Acct_454', 'Acct_460', 'Acct_564A']

#Use list of accounts of interest to create master dataframe of accounts for analysis by referencing against the full NCUA account list
#Create custom 'account key' field that will match with fields in needed data tables
NCUAIntAccts = pd.DataFrame(NCUAIntAcctsList)
NCUAIntAccts = NCUAIntAccts.rename(columns = {0 : 'AcctsOfInterest'})
NCUAIntAccts = NCUAIntAccts.merge(NCUAdesc, how = 'left', left_on = 'AcctsOfInterest', right_on = 'Account')
NCUAIntAccts = NCUAIntAccts.drop(columns = ['Annualize', 'CBSAccount', 'FPRAccount', 'STATAcctMain', 'STATAcctState', 'STATAcctGraph', 'STATAcctTable8'])
NCUAIntAccts['AccountKey'] = NCUAIntAccts['Account'].str.replace('Acct', 'ACCT')
NCUAIntAccts['AccountKey'] = NCUAIntAccts['AccountKey'].str.replace('ACCT_661A', 'Acct_661A')
NCUAIntAccts

,AcctsOfInterest,Account,AcctName,AcctDesc,TableName,STATUS,AccountKey
0,ACCT_AS0009,ACCT_AS0009,Total Cash and Other Deposits,Total Cash and Other Deposits,FS220P,Active,ACCT_AS0009
1,ACCT_AS0013,ACCT_AS0013,Total Investment Securities,Total Investment Securities,FS220P,Active,ACCT_AS0013
2,Acct_025B,Acct_025B,Total amount of Loans and Leases,Total amount of Loans and Leases,FS220,Active,ACCT_025B
3,Acct_010,Acct_010,TOTAL ASSETS,TOTAL ASSETS,FS220,Active,ACCT_010
4,Acct_LI0069,Acct_LI0069,Total Liabilities,Total Liabilities,FS220N,Active,ACCT_LI0069
5,Acct_014,Acct_014,"TOTAL LIABILITIES, SHARES, AND EQUITY","TOTAL LIABILITIES, SHARES, AND EQUITY",FS220A,Active,ACCT_014
6,Acct_115,Acct_115,Total Interest Income,Total Interest Income,FS220A,Active,ACCT_115
7,ACCT_IS0010,ACCT_IS0010,Net Interest Income,Net Interest Income,FS220P,Active,ACCT_IS0010
8,Acct_117,Acct_117,Total Non-Interest Income,Total Non-Interest Income,FS220A,Active,ACCT_117
9,Acct_671,Acct_671,Total Non-Interest Expense (Sum of items 18-27),Sum of all operating expenses.,FS220,Active,ACCT_671


In [40]:
#Summarize data tables needed
NCUATablesNeeded = pd.unique(NCUAIntAccts.TableName).tolist()
NCUATablesNeeded

['FS220P', 'FS220', 'FS220N', 'FS220A']

In [41]:
#Create account lists for each queried table
FS220PAccts = NCUAIntAccts[NCUAIntAccts.TableName == 'FS220P']['AccountKey'].tolist()
FS220PAccts.append('CU_NUMBER')

FS220Accts = NCUAIntAccts[NCUAIntAccts.TableName == 'FS220']['AccountKey'].tolist()
FS220Accts.append('CU_NUMBER')

FS220NAccts = NCUAIntAccts[NCUAIntAccts.TableName == 'FS220N']['AccountKey'].tolist()
FS220NAccts.append('CU_Number')

FS220AAccts = NCUAIntAccts[NCUAIntAccts.TableName == 'FS220A']['AccountKey'].tolist()
FS220AAccts.append('CU_NUMBER')

FS220Accts

['ACCT_025B', 'ACCT_010', 'ACCT_671', 'ACCT_083', 'CU_NUMBER']

In [42]:
#data table filtering and consolidation

FS220P = FS220P[FS220PAccts]
FS220 = FS220[FS220Accts]
FS220N = FS220N[FS220NAccts]
FS220A = FS220A[FS220AAccts]
FOICU = FOICU[['CU_NUMBER', 'CYCLE_DATE', 'CU_TYPE', 'CU_NAME']]

NCUARawData = FOICU.merge(FS220P, how = 'left', left_on = 'CU_NUMBER', right_on = 'CU_NUMBER')
NCUARawData = NCUARawData.merge(FS220, how = 'left', left_on = 'CU_NUMBER', right_on = 'CU_NUMBER')
NCUARawData = NCUARawData.merge(FS220N, how = 'left', left_on = 'CU_NUMBER', right_on = 'CU_Number')
NCUARawData = NCUARawData.merge(FS220A, how = 'left', left_on = 'CU_NUMBER', right_on = 'CU_NUMBER')
NCUARawData = NCUARawData.drop(columns = 'CU_Number')
NCUARawData

,CU_NUMBER,CYCLE_DATE,CU_TYPE,CU_NAME,ACCT_AS0009,ACCT_AS0013,ACCT_IS0010,ACCT_025B,ACCT_010,ACCT_671,ACCT_083,ACCT_LI0069,ACCT_014,ACCT_115,ACCT_117,Acct_661A,ACCT_452,ACCT_454,ACCT_460,ACCT_564A
0,1,6/30/2024 0:00:00,1,MORRIS SHEPPARD TEXARKANA,2167913,0,217665,8821970,11109125,196035,1259,9767809,11109125,285728,38664,48799,325,1259,1762,3
1,6,6/30/2024 0:00:00,1,THE NEW ORLEANS FIREMEN'S,32481020,32976567,6065933,194109534,274014841,7299810,28845,253454955,274014841,7726443,2197410,-136579,14385,33688,82811,92
2,12,6/30/2024 0:00:00,1,FRANKLIN TRUST,17469321,6221524,1474104,27695075,60169418,2365075,9073,55410098,60169418,1665969,1229941,144609,6210,14580,22045,25
3,13,6/30/2024 0:00:00,1,EFCU FINANCIAL,59620126,302011,12025216,977889001,1074739920,12021058,66652,987001776,1074739920,27297633,5988517,4721598,33322,72948,117641,126
4,16,6/30/2024 0:00:00,1,WOODMEN,5662595,0,160936,3816079,9617599,182804,886,8341687,9617599,184440,28750,6882,341,1114,1529,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4626,97108,6/30/2024 0:00:00,3,NORTHWEST COMMUNITY,11781012,7927968,645282,43900214,66460005,809757,2656,61041125,66460005,1352783,76258,-85217,841,2914,4672,8
4627,97109,6/30/2024 0:00:00,3,RIVER VALLEY,39780389,48383194,7151856,362790254,470938640,6958610,25905,424987402,470938640,11508316,2274508,1692355,13318,32305,51638,80
4628,97110,6/30/2024 0:00:00,3,ROCKY MOUNTAIN,34451732,41522702,6054937,269680844,374622068,6994466,24747,343541453,374622068,9053513,2348882,864853,15021,25242,50370,75
4629,97111,6/30/2024 0:00:00,3,ABBEY,19032831,13180481,2937846,122734916,164782804,2881900,13007,154831632,164782804,4210905,1024461,911498,7608,14667,24731,41


### FDIC Data

In [43]:
#Import FDIC field descriptors and API instructions
FDICdesc = pd.read_excel(open('All Financial Reports.xlsx', 'rb'), sheet_name = 'Reference-Variables&Definitions')
FDICdesc

,Variable,Title,Definition
0,ABCUBKR,Unused Commitments To Provide Liquidity To Con...,"( YTD, % ) Conduits Sponsored by the Bank - Un..."
1,ABCUOTH,"Unused Commitments To Provide Liquidity, Unrel...","( YTD, $ ) Conduits Sponsored by Other Unrelat..."
2,ABCUOTHR,"Unused Commitments To Provide Liquidity, Unrel...","( YTD, % ) Conduits Sponsored by Other Unrelat..."
3,ABCXBK,Maximum Credit Exposure Asset-Backed Commercia...,"( YTD, $ ) Conduits Sponsored by the Bank Maxi..."
4,ABCXBKR,Maximum Credit Exposure Asset-Backed Commercia...,"( YTD, % ) Conduits Sponsored by the Bank Maxi..."
...,...,...,...
2328,UNINUM,FDIC's unique number,A unique number for the bank or savings instit...
2329,VOLIAB,Volatile Liabilities,"( YTD, $ ) Volatile liabilities on a consolida..."
2330,VOLIABR,Volatile Liabilities,"( YTD, % ) Volatile liabilities on a consolida..."
2331,WEBADDR,Primary Internet Web Address,The primary internet web address is the public...


In [44]:
FDICIntAcctsList = ['BKCLASS', 'CERT', 'NAMEFULL', 'REPDTE', 'CHBAL', 'SC', 'LNLSNET', 'ASSET', 'LIAB', 'LIABEQ', 'INTINC', 'NIM', 'NONII', 'NONIX', 'NETINC', 'DEPLGB', 'DEPSMB', 'NUMEMP']
FDICIntAccts = pd.DataFrame(FDICIntAcctsList)
FDICIntAccts = FDICIntAccts.rename(columns = {0 : 'AcctsOfInterest'})
FDICIntAccts = FDICIntAccts.merge(FDICdesc, how = 'left', left_on = 'AcctsOfInterest', right_on = 'Variable')
FDICIntAccts

,AcctsOfInterest,Variable,Title,Definition
0,BKCLASS,BKCLASS,Institution Class,A classification code assigned by the FDIC bas...
1,CERT,CERT,FDIC Certificate #,A unique number assigned by the FDIC used to i...
2,NAMEFULL,NAMEFULL,Institution Name at Time of Reporting,Institution Name at Time of Reporting
3,REPDTE,REPDTE,Report Date,The last day of the financial reporting period...
4,CHBAL,CHBAL,Cash and Balances due from Depository Institut...,"( YTD, $ ) Total cash and balances due from de..."
5,SC,SC,Total Securities,"( YTD, $ ) Total rsecurities: the sum of held-..."
6,LNLSNET,LNLSNET,Net Loans and Leases,"( YTD, $ ) Net Loans and Leases"
7,ASSET,ASSET,Total Assets,"( YTD, $ ) The sum of all assets owned by the ..."
8,LIAB,LIAB,Total Liabilities,"( YTD, $ ) Deposits and other borrowings, subo..."
9,LIABEQ,LIABEQ,Total Liabilities & Capital,"( YTD, $ ) Total liabilities, limited-life pre..."


In [45]:
#Write URL API string to query desired data from FDIC website
FDICDate = '20240630'

FDICAPIScript = str('https://banks.data.fdic.gov/api/financials?filters=ACTIVE%3A1%20AND%20%21%28BKCLASS%3ANC%29%20AND%20REPDTE%3A' + FDICDate + '&fields=')

for Acct in FDICIntAcctsList:
    FDICAPIScript = str(FDICAPIScript + Acct + '%2C')

FDICAPIScript = FDICAPIScript[: - 3]

FDICAPIScript = str(FDICAPIScript + '&sort_by=REPDTE&sort_order=DESC&limit=10000&offset=0&format=csv&download=true&filename=data_file')

In [46]:
#Import downloaded FDIC dataset
FDICRawData = pd.read_csv(FDICAPIScript)
FDICRawData

,ASSET,BKCLASS,CERT,CHBAL,DEPLGB,DEPSMB,ID,INTINC,LIAB,LIABEQ,LNLSNET,NAMEFULL,NETINC,NIM,NONII,NONIX,NUMEMP,REPDTE,SC
0,232555,NM,10004,5507,85,10704,10004_20240630,6579.0,209892,232555,192882,ERGO BANK,762.0,4028.0,418.0,3467.0,42.0,20240630,21910
1,441130,NM,10011,6826,164,16799,10011_20240630,10575.0,419962,441130,317497,WOODFORD STATE BANK,1484.0,6203.0,728.0,5301.0,73.0,20240630,91636
2,200342,NM,10012,19315,96,6978,10012_20240630,4869.0,177746,200342,111263,THE PORTAGE COUNTY BANK,1142.0,3283.0,366.0,2477.0,37.0,20240630,61015
3,192915,NM,10015,3689,68,6629,10015_20240630,5972.0,170054,192915,165952,SECURITY BANK,1255.0,3520.0,333.0,2174.0,24.0,20240630,6087
4,2758478,N,10044,200466,857,80033,10044_20240630,73503.0,2286528,2758478,1601542,NATIONAL EXCHANGE BANK AND TRUST,18439.0,48676.0,7260.0,27135.0,374.0,20240630,833509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4543,88724,SM,9986,7863,19,4686,9986_20240630,1799.0,74142,88724,48844,FARMERS & MERCHANTS BANK OF CRAIG COUNTY,347.0,1351.0,89.0,933.0,15.0,20240630,29626
4544,541958,NM,9997,28712,164,18090,9997_20240630,11461.0,516204,541958,373951,JEFFERSON SECURITY BANK,1542.0,6431.0,1122.0,5490.0,78.0,20240630,114297
4545,218835,NM,9998,15077,80,11201,9998_20240630,5482.0,199858,218835,152387,CAPON VALLEY BANK,804.0,4019.0,625.0,3518.0,47.0,20240630,38263
4546,101066,SM,999,27605,47,4324,999_20240630,2108.0,92254,101066,24443,G. W. JONES EXCHANGE BANK,496.0,1678.0,586.0,1646.0,20.0,20240630,47210


### NCUA Data Transformation

In [47]:
NCUAIntAccts['AccountKey'].to_list()

['ACCT_AS0009',
 'ACCT_AS0013',
 'ACCT_025B',
 'ACCT_010',
 'ACCT_LI0069',
 'ACCT_014',
 'ACCT_115',
 'ACCT_IS0010',
 'ACCT_117',
 'ACCT_671',
 'Acct_661A',
 'ACCT_083',
 'ACCT_452',
 'ACCT_454',
 'ACCT_460',
 'ACCT_564A']

In [48]:
#Rename and standardize column names
NCUARevData = NCUARawData.rename(columns = {
    'ACCT_AS0009' : 'Cash & Deposits',
 'ACCT_AS0013' : 'Investments',
 'ACCT_025B' : 'Loans & Leases',
 'ACCT_010' : 'Assets',
 'ACCT_LI0069' : 'Liabilities',
 'ACCT_014' : 'Liabilities & Equity',
 'ACCT_115' : 'Interest Income',
 'ACCT_IS0010' : 'Net Interest Income',
 'ACCT_117' : 'Non-Interest Income',
 'ACCT_671' : 'Non-Interest Expense',
 'Acct_661A' : 'Net Income',
 'ACCT_083' : 'Members',
 'ACCT_452' : 'Share Draft Accounts',
 'ACCT_454' : 'Regular Share Accounts',
 'ACCT_460' : 'Deposit Accounts',
 'ACCT_564A' : 'Employees',
 'CU_NUMBER' : 'Charter/Cert',
 'CYCLE_DATE' : 'Report Date',
 'CU_TYPE' : 'Inst Type Code',
 'CU_NAME' : 'Inst Name'})

#Add a column to specify between credit union, bank, savings bank. For use in merged table.
NCUARevData['Inst Type'] = 'Credit Union'

#Combine Inst Type with Charter/Cert to create a true unique identifier. For use in merged table.
NCUARevData['Charter/Cert'] = NCUARevData['Inst Type'].astype(str) + '_' + NCUARevData['Charter/Cert'].astype(str)

#Convert all financial data to millions to provide proper scale for analysis
NCUAConvScale = 1000000

NCUARevData[['Cash & Deposits', 'Investments', 'Loans & Leases', 'Assets', 'Liabilities', 'Liabilities & Equity', 'Interest Income', 'Net Interest Income', 'Non-Interest Income', 'Non-Interest Expense', 'Net Income']] = NCUARevData[[
    'Cash & Deposits', 'Investments', 'Loans & Leases', 'Assets', 'Liabilities', 'Liabilities & Equity', 'Interest Income', 'Net Interest Income', 'Non-Interest Income', 'Non-Interest Expense', 'Net Income']] / NCUAConvScale

#Round all figures to the nearest dollar
NCUARevData = NCUARevData.round(0)
NCUARevData

,Charter/Cert,Report Date,Inst Type Code,Inst Name,Cash & Deposits,Investments,Net Interest Income,Loans & Leases,Assets,Non-Interest Expense,...,Liabilities,Liabilities & Equity,Interest Income,Non-Interest Income,Net Income,Share Draft Accounts,Regular Share Accounts,Deposit Accounts,Employees,Inst Type
0,Credit Union_1,6/30/2024 0:00:00,1,MORRIS SHEPPARD TEXARKANA,2.0,0.0,0.0,9.0,11.0,0.0,...,10.0,11.0,0.0,0.0,0.0,325,1259,1762,3,Credit Union
1,Credit Union_6,6/30/2024 0:00:00,1,THE NEW ORLEANS FIREMEN'S,32.0,33.0,6.0,194.0,274.0,7.0,...,253.0,274.0,8.0,2.0,-0.0,14385,33688,82811,92,Credit Union
2,Credit Union_12,6/30/2024 0:00:00,1,FRANKLIN TRUST,17.0,6.0,1.0,28.0,60.0,2.0,...,55.0,60.0,2.0,1.0,0.0,6210,14580,22045,25,Credit Union
3,Credit Union_13,6/30/2024 0:00:00,1,EFCU FINANCIAL,60.0,0.0,12.0,978.0,1075.0,12.0,...,987.0,1075.0,27.0,6.0,5.0,33322,72948,117641,126,Credit Union
4,Credit Union_16,6/30/2024 0:00:00,1,WOODMEN,6.0,0.0,0.0,4.0,10.0,0.0,...,8.0,10.0,0.0,0.0,0.0,341,1114,1529,2,Credit Union
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4626,Credit Union_97108,6/30/2024 0:00:00,3,NORTHWEST COMMUNITY,12.0,8.0,1.0,44.0,66.0,1.0,...,61.0,66.0,1.0,0.0,-0.0,841,2914,4672,8,Credit Union
4627,Credit Union_97109,6/30/2024 0:00:00,3,RIVER VALLEY,40.0,48.0,7.0,363.0,471.0,7.0,...,425.0,471.0,12.0,2.0,2.0,13318,32305,51638,80,Credit Union
4628,Credit Union_97110,6/30/2024 0:00:00,3,ROCKY MOUNTAIN,34.0,42.0,6.0,270.0,375.0,7.0,...,344.0,375.0,9.0,2.0,1.0,15021,25242,50370,75,Credit Union
4629,Credit Union_97111,6/30/2024 0:00:00,3,ABBEY,19.0,13.0,3.0,123.0,165.0,3.0,...,155.0,165.0,4.0,1.0,1.0,7608,14667,24731,41,Credit Union


### FDIC Data Transformation

In [49]:
FDICIntAccts['AcctsOfInterest'].to_list()

['BKCLASS',
 'CERT',
 'NAMEFULL',
 'REPDTE',
 'CHBAL',
 'SC',
 'LNLSNET',
 'ASSET',
 'LIAB',
 'LIABEQ',
 'INTINC',
 'NIM',
 'NONII',
 'NONIX',
 'NETINC',
 'DEPLGB',
 'DEPSMB',
 'NUMEMP']

In [50]:
#Rename and standardize column names
FDICRevData = FDICRawData.rename(columns = {
    'BKCLASS' : 'Inst Type Code',
 'CERT' : 'Charter/Cert',
 'NAMEFULL' : 'Inst Name',
 'REPDTE' : 'Report Date',
 'CHBAL' : 'Cash & Deposits',
 'SC' : 'Investments',
 'LNLSNET' : 'Loans & Leases',
 'ASSET' : 'Assets',
 'LIAB' : 'Liabilities',
 'LIABEQ' : 'Liabilities & Equity',
 'INTINC' : 'Interest Income',
 'NIM' : 'Net Interest Income',
 'NONII' : 'Non-Interest Income',
 'NONIX' : 'Non-Interest Expense',
 'NETINC' : 'Net Income',
 'DEPLGB' : '> $100K Accounts',
 'DEPSMB' : '< $100K Accounts',
 'NUMEMP' : 'Employees'})

#Add a column to specify between credit union, bank, savings bank. For use in merged table.
#Default inst type to bank
FDICRevData['Inst Type'] = 'Bank'

#Check if inst type should be changed to savings bank or CU
FDICRevData.loc[(FDICRevData['Inst Type Code'] == 'SB') | (FDICRevData['Inst Type Code'] == 'SI') | (FDICRevData['Inst Type Code'] == 'SL') | (FDICRevData['Inst Type Code'] == 'NS'), 'Inst Type'] = 'Savings Bank'
FDICRevData.loc[(FDICRevData['Inst Type Code'] == 'CU'), 'Inst Type'] = 'Credit Union'

#Combine Inst Type with Charter/Cert to create a true unique identifier. For use in merged table.
FDICRevData['Charter/Cert'] = FDICRevData['Inst Type'].astype(str) + '_' + FDICRevData['Charter/Cert'].astype(str)

#Create total deposits field using >$100K accounts and <$100K accounts fields
FDICRevData['Deposit Accounts'] = FDICRevData['< $100K Accounts'] + FDICRevData['> $100K Accounts']

#Drop unneeded 'ID' field
FDICRevData = FDICRevData.drop(columns = {'ID'})

#Convert all financial data to millions to provide proper scale for analysis
FDICConvScale = 1000

FDICRevData[['Cash & Deposits', 'Investments', 'Loans & Leases', 'Assets', 'Liabilities', 'Liabilities & Equity', 'Interest Income', 'Net Interest Income', 'Non-Interest Income', 'Non-Interest Expense', 'Net Income']] = FDICRevData[[
    'Cash & Deposits', 'Investments', 'Loans & Leases', 'Assets', 'Liabilities', 'Liabilities & Equity', 'Interest Income', 'Net Interest Income', 'Non-Interest Income', 'Non-Interest Expense', 'Net Income']] / FDICConvScale

#Round all figures to the nearest dollar
FDICRevData = FDICRevData.round(0)
FDICRevData

,Assets,Inst Type Code,Charter/Cert,Cash & Deposits,> $100K Accounts,< $100K Accounts,Interest Income,Liabilities,Liabilities & Equity,Loans & Leases,Inst Name,Net Income,Net Interest Income,Non-Interest Income,Non-Interest Expense,Employees,Report Date,Investments,Inst Type,Deposit Accounts
0,233.0,NM,Bank_10004,6.0,85,10704,7.0,210.0,233.0,193.0,ERGO BANK,1.0,4.0,0.0,3.0,42.0,20240630,22.0,Bank,10789
1,441.0,NM,Bank_10011,7.0,164,16799,11.0,420.0,441.0,317.0,WOODFORD STATE BANK,1.0,6.0,1.0,5.0,73.0,20240630,92.0,Bank,16963
2,200.0,NM,Bank_10012,19.0,96,6978,5.0,178.0,200.0,111.0,THE PORTAGE COUNTY BANK,1.0,3.0,0.0,2.0,37.0,20240630,61.0,Bank,7074
3,193.0,NM,Bank_10015,4.0,68,6629,6.0,170.0,193.0,166.0,SECURITY BANK,1.0,4.0,0.0,2.0,24.0,20240630,6.0,Bank,6697
4,2758.0,N,Bank_10044,200.0,857,80033,74.0,2287.0,2758.0,1602.0,NATIONAL EXCHANGE BANK AND TRUST,18.0,49.0,7.0,27.0,374.0,20240630,834.0,Bank,80890
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4543,89.0,SM,Bank_9986,8.0,19,4686,2.0,74.0,89.0,49.0,FARMERS & MERCHANTS BANK OF CRAIG COUNTY,0.0,1.0,0.0,1.0,15.0,20240630,30.0,Bank,4705
4544,542.0,NM,Bank_9997,29.0,164,18090,11.0,516.0,542.0,374.0,JEFFERSON SECURITY BANK,2.0,6.0,1.0,5.0,78.0,20240630,114.0,Bank,18254
4545,219.0,NM,Bank_9998,15.0,80,11201,5.0,200.0,219.0,152.0,CAPON VALLEY BANK,1.0,4.0,1.0,4.0,47.0,20240630,38.0,Bank,11281
4546,101.0,SM,Bank_999,28.0,47,4324,2.0,92.0,101.0,24.0,G. W. JONES EXCHANGE BANK,0.0,2.0,1.0,2.0,20.0,20240630,47.0,Bank,4371


### Combine NCUA and FDIC data

In [51]:
#Combine NCUA and FDIC datasets into a single dataframe
frames = [NCUARevData, FDICRevData]
FIData = pd.concat(frames)

#Drop fields that don't apply to the merged dataset
FIData = FIData.drop(columns = {'Share Draft Accounts', 'Regular Share Accounts', '> $100K Accounts', '< $100K Accounts', 'Members'})

#Add ratio/metric fields
FIData['Revenue'] = FIData['Interest Income'] + FIData['Non-Interest Income']
FIData['Cash to Assets'] = FIData['Cash & Deposits'] / FIData['Assets']
FIData['Inv to Assets'] = FIData['Investments'] / FIData['Assets']
FIData['Loans to Assets'] = FIData['Loans & Leases'] / FIData['Assets']
FIData['Interest to Revenue'] = FIData['Interest Income'] / FIData['Revenue']
FIData['Non-Interest to Revenue'] = FIData['Non-Interest Income'] / FIData['Revenue']
FIData['Net Income to Revenue'] = FIData['Net Income'] / FIData['Revenue']
FIData['Return on Assets'] = FIData['Net Income'] / FIData['Assets']

FIData

,Charter/Cert,Report Date,Inst Type Code,Inst Name,Cash & Deposits,Investments,Net Interest Income,Loans & Leases,Assets,Non-Interest Expense,...,Employees,Inst Type,Revenue,Cash to Assets,Inv to Assets,Loans to Assets,Interest to Revenue,Non-Interest to Revenue,Net Income to Revenue,Return on Assets
0,Credit Union_1,6/30/2024 0:00:00,1,MORRIS SHEPPARD TEXARKANA,2.0,0.0,0.0,9.0,11.0,0.0,...,3.0,Credit Union,0.0,0.181818,0.000000,0.818182,NaN,NaN,NaN,0.000000
1,Credit Union_6,6/30/2024 0:00:00,1,THE NEW ORLEANS FIREMEN'S,32.0,33.0,6.0,194.0,274.0,7.0,...,92.0,Credit Union,10.0,0.116788,0.120438,0.708029,0.800000,0.200000,-0.000000,-0.000000
2,Credit Union_12,6/30/2024 0:00:00,1,FRANKLIN TRUST,17.0,6.0,1.0,28.0,60.0,2.0,...,25.0,Credit Union,3.0,0.283333,0.100000,0.466667,0.666667,0.333333,0.000000,0.000000
3,Credit Union_13,6/30/2024 0:00:00,1,EFCU FINANCIAL,60.0,0.0,12.0,978.0,1075.0,12.0,...,126.0,Credit Union,33.0,0.055814,0.000000,0.909767,0.818182,0.181818,0.151515,0.004651
4,Credit Union_16,6/30/2024 0:00:00,1,WOODMEN,6.0,0.0,0.0,4.0,10.0,0.0,...,2.0,Credit Union,0.0,0.600000,0.000000,0.400000,NaN,NaN,NaN,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4543,Bank_9986,20240630,SM,FARMERS & MERCHANTS BANK OF CRAIG COUNTY,8.0,30.0,1.0,49.0,89.0,1.0,...,15.0,Bank,2.0,0.089888,0.337079,0.550562,1.000000,0.000000,0.000000,0.000000
4544,Bank_9997,20240630,NM,JEFFERSON SECURITY BANK,29.0,114.0,6.0,374.0,542.0,5.0,...,78.0,Bank,12.0,0.053506,0.210332,0.690037,0.916667,0.083333,0.166667,0.003690
4545,Bank_9998,20240630,NM,CAPON VALLEY BANK,15.0,38.0,4.0,152.0,219.0,4.0,...,47.0,Bank,6.0,0.068493,0.173516,0.694064,0.833333,0.166667,0.166667,0.004566
4546,Bank_999,20240630,SM,G. W. JONES EXCHANGE BANK,28.0,47.0,2.0,24.0,101.0,2.0,...,20.0,Bank,3.0,0.277228,0.465347,0.237624,0.666667,0.333333,0.000000,0.000000


In [52]:
FIData.to_csv('FIData.csv')

### Data Cleaning

In [53]:
# #Remove records that have zero or less than zero in fields where it doesn't make sense to have those results
# RemoveRecs = FIData[(FIData['Assets'] <= 0) | (FIData['Cash & Deposits'] < 0) | (FIData['Revenue'] < 0) | (FIData['Net Income'] < 0) | (FIData['Net Interest Income'] < 0) | (FIData['Non-Interest Income'] < 0)].index
# FIData.drop(RemoveRecs, inplace = True)

# #Drop NA records. Assume incorrect records
# FIData = FIData.dropna()

# #Assign significant bank outliers to 'Super Bank' inst type
# FIData.loc[(FIData['Charter/Cert'] == 'Bank_628') | (FIData['Charter/Cert'] == 'Bank_3510') | (FIData['Charter/Cert'] == 'Bank_3511') | (FIData['Charter/Cert'] == 'Bank_7213'), 'Inst Type'] = 'Super Bank'

# #Review our overall stats again to determine if our data is clean enough to proceed to analysis
# FIDataStats = FIData.describe()
# FIDataStats = FIDataStats.style.format(precision = 2, thousands = ',')
# FIDataStats